problemas:

- filtrar noticias muito parecidas

- 1. **fazer NER/sentiment, dois iguais, ver se é 80% igual?**

- 2. selecionar 4 frases aleatórias ou contar palavras, e se for igual pronto?

- 3. "bbiggest substring" - se >80%, é igual ?

- que modelo usar para Sentiment Analysis ?


tentar NER + Sentiment Analysis

---

to do now:

- ~~save all data in a single column or something different inside the parquet~~

- ~~add news source to dicionary~~

- ~~convert timestamp to YYYYMM because ts is from archived, not the new itself~~

- ~~reorder dicionary~~

In [247]:
import pandas as pd
from datetime import datetime

---

**merged api columns into news**

In [248]:
df = pd.read_parquet("data03.parquet").map(lambda x: list(x))
df['news'] = df.iloc[:, 1:].sum(axis=1)
df_filtered = df.iloc[:, [0, -1]]
df_filtered

,aliases,news
companies,,
Banco Comercial Português,"[Banco Comercial Português, BCP]",[{'ExtractedText': 'DN 13 de Setembro de 200...
Galp Energia,"[Galp Energia, GALP]",[{'ExtractedText': 'RTP Galp reforça posição n...
EDP,"[EDP, Energias de Portugal, Electricidade de P...",[{'ExtractedText': 'DN-Sinteses Negocios 9 de ...
Sonae,"[Sonae, SON]",[{'ExtractedText': 'DN-Sinteses 5 de Março de ...
Mota-Engil,"[Mota-Engil, EGL]",[{'ExtractedText': 'RTP Lucro da Mota-Engil so...


In [249]:
df_filtered.map(len)

,aliases,news
companies,,
Banco Comercial Português,2,1284
Galp Energia,2,1338
EDP,3,1456
Sonae,2,899
Mota-Engil,2,459


**convert timestamp +  add sources + reorder dictionary**

In [250]:
def tstampANDsourceANDreorderdict(lista):
    new_list = []
    noticias = pd.read_csv("noticias.csv")
    for req in lista:
        # news source
        linkToArchive = req["linkToArchive"]
        foundSource = False
        for index, row in noticias.iterrows():
            if row.iloc[0] in linkToArchive:
                req["newsSource"] = row.iloc[1]
                foundSource = True
                break
            else:
                pass
        if not foundSource:
            req["newsSource"] = "unknown"
        # timestamp
        req["tstamp"] = req["tstamp"][:6]
        # reorder dict
        ordered_keys = ["tstamp", "newsSource", "newsProbability", "linkToArchive", "ExtractedText"]
        reordered_dict = {key: req[key] for key in ordered_keys}
        # SAVE
        new_list.append(reordered_dict)
    return new_list

df_filtered.loc[:, "news"] = df_filtered["news"].map(lambda x: tstampANDsourceANDreorderdict(x))

In [251]:
df_filtered.iloc[0,-1][0]

{'tstamp': '200109',
 'newsSource': 'Diário de Notícias',
 'newsProbability': 0.933,
 'linkToArchive': 'https://arquivo.pt/wayback/20010913052557/http://www.dn.pt/int/13p4x.htm',
 'ExtractedText': 'DN \xa0 13 de Setembro de 2001 Intalações do BCP servem de refúgio As instalações do Banco Comercial Português, em Wall Street, foram utilizadas como refúgio por dezenas de pessoas depois dos atentados ao World Trade Center. Nenhum dos 48 funcionários daquela sucursal bancário ficou ferido e os dois trabalhadores que ainda não tinham comunicado com empresa fizeram-no na madrugada de ontem. Voo TAP para Newark chega hoje O voo 1331 da TAP que partiu terça-feira de Lisboa para Newark, nos Estados Unidos, deve chegar hoje ao seu destino. Segundo anunciou a porta-voz da companhia aérea, o avião foi direccionado para a cidade de Halifax, no Canadá, quando os EUA encerraram todos os seus aeroportos a vôos comerciais que se dirigissem para o território. EUA agradecem apoio português O representante

---

In [252]:
from fuzzywuzzy import fuzz

def find_similar_texts(texts, threshold=80):
    similar_texts = []
    
    # Iterate through each text
    for i in range(1, len(texts)):
        # Compare the current text (texts[i]) with all previous ones (texts[:i])
        for j in range(i):
            similarity = fuzz.ratio(texts[i], texts[j])
            # Print similarity score for debugging
            print(f"Comparing: '{texts[i]}' with '{texts[j]}', Similarity: {similarity}%")
            if similarity >= threshold:
                similar_texts.append((texts[i], texts[j], similarity))
    
    return similar_texts

# Example list of texts
texts = [
    "This is the first text.",
    "This is a simple text.",
    "Another text example.",
    "Yet another sample."
]

# Run the function
results = find_similar_texts(texts, threshold=80)

# Print results
if results:
    for res in results:
        print(f"Text 1: {res[0]}\nText 2: {res[1]}\nSimilarity: {res[2]}%\n")
else:
    print("No similar texts found above the threshold.")


Comparing: 'This is a simple text.' with 'This is the first text.', Similarity: 71%
Comparing: 'Another text example.' with 'This is the first text.', Similarity: 45%
Comparing: 'Another text example.' with 'This is a simple text.', Similarity: 37%
Comparing: 'Yet another sample.' with 'This is the first text.', Similarity: 38%
Comparing: 'Yet another sample.' with 'This is a simple text.', Similarity: 44%
Comparing: 'Yet another sample.' with 'Another text example.', Similarity: 65%
No similar texts found above the threshold.
